# analysis

## ToDo

- ~~Importing more than one json file~~
- ~~How to group by subjects models and movement~~
- ~~Model specific confusion rat~~
- How to manage or represent all this data
- ...

## data structure

Grouping
- subjects
- models
    - type overall
    - individual configuration
- movement
    - type overall
    - with(out) contact

## loading data
json file needs to be normalized

In [ ]:
import numpy as np
import pandas as pd
import json, os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def parse_modelname(x='model(dmp)-dataset(return-bottle-hold)-npsi(90)-hold(1)'):
    x = x.split('-dataset(')
    typ = x[1].split(')',1)
    typ = typ[1]
    x = x[0].split('model(')
    x = x[1][:-1]
    return x + typ

In [ ]:
path_to_json = '/users/hieu/git/turing_test_walking/data'

json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
json_files.pop()
first_file = json_files.pop()
with open(first_file,'r') as f:
    one = json.loads(f.read())
data = pd.json_normalize(one)

#workaround for concatenating unnormalized data
for file in json_files:
    with open(file,'r') as f:
        one = json.loads(f.read())
    load = pd.json_normalize(one)
    data = pd.concat([data,load])

  ## parsing

In [ ]:
# filtering test parts and creating new index
df = data[data['test_part'] == 'response'].reset_index()
df = df.fillna(value='99')
df = df[df['key_press'] != '99'].reset_index()

# this line below for filtering attention checks overall out -> uncomment for using it
# df = data[(data['test_part'] == 'response') & (data['.att_check'] == False)].reset_index()

# must be integers for comparing
df['key_press'] = df['key_press'].astype(int)
df['test_stimulus.position'] = df['test_stimulus.position'].astype(int)

# renaming columns
df = df.rename(columns={"test_stimulus.left": "left",
                        "test_stimulus.right": "right",
                        "test_stimulus.att_check": "att_check",
                        "test_stimulus.position": "position",
                        "key_press":"pressed"})

In [ ]:
df = df[['sona','subject','rt','att_check','pressed','position','left','right']]

In [ ]:
# without copy() there is an error when replacing strings
d = df.copy()

In [ ]:
# removing unnecessary strings
d['left'] = d['left'].str.replace('bvh/vr_prediction_models/','')
d['left'] = d['left'].str.replace('bvh/','')
d['left'] = d['left'].str.replace('/final-lines.txt','')
d['left'] = d['left'].str.replace('-lines.txt','')
d['right'] = d['right'].str.replace('bvh/vr_prediction_models/','')
d['right'] = d['right'].str.replace('bvh/','')
d['right'] = d['right'].str.replace('/final-lines.txt','')
d['right'] = d['right'].str.replace('-lines.txt','')

In [ ]:
d['pressed']

In [ ]:
# recode keypress
num = 0
arr = []
for i in d['pressed']:
    if i == 70:
        arr.append(0)
        
    elif i == 74:
        arr.append(1)
d['pressed'] = arr

In [ ]:
d['position']

In [ ]:
# adding movement column for better grouping
num = 0
arr = []

for i in d['position']:
    if i == 1:
        arr.append(d['right'][num])
    else:
        arr.append(d['left'][num])
    num += 1

movement = []
for i in arr:
    x = i.split('-')
    movement.append('-'.join(x[:-1]))
d['movement'] = movement

In [ ]:
# adding model column for easier grouping by
num = 0
arr = []
for i in d['position']:
    if i == 1:
        arr.append(d['left'][num])
    else:
        arr.append(d['right'][num])
    num += 1
d['model_set'] = arr

In [ ]:
d

**old code for model column**

arr = []
for i in d['model_set']:
    string = i.split('-')
    string = string[0].replace('model(','')
    string = string.replace(')','')
    arr.append(string)
d['model'] = arr

In [ ]:
# creating model column for better grouping by models
arr = []
for i in d['model_set']:
    if i.find('tmp')>0:
        arr.append('tmp')
    elif i.find('dmp')>0:
        arr.append('dmp')
    elif i.find('MAP')>0:
        arr.append('vcgpdm-map')
    elif i.find('ELBO')>0:
        arr.append('vcgpdm-elbo')
        
d['model'] = arr

In [ ]:
arr = []
for i in d['model_set']:
    arr.append(parse_modelname(i))
d['model_set'] = arr

In [ ]:
# adding performance score
num = 0
arr = []
for i in d['pressed']:
    if i == d['position'][num]:
        arr.append(1)
    else:
        arr.append(0)
    num += 1
d['p_score'] = arr

#### Taking only Sona

In [ ]:
d=d[d['sona']!='99']
d=d[d['sona']!='THNS']

In [ ]:
set(d['sona'])

## analysis
### overall confusion rate
#### with all attention checks included

In [ ]:
correct = 0
for i in d['p_score']:
    correct += i
confusion_rate = (len(d) - correct)/len(d)
print(confusion_rate)

#### without attention checks

In [ ]:
no_check = d[d['att_check']==False]

In [ ]:
correct = 0
for i in no_check['p_score']:
        correct += i
confusion_rate = (len(no_check) - correct)/len(no_check)
print(confusion_rate)

In [ ]:
no_check.shape

### checking attention
#### if confusion rate is zero -> good

In [ ]:
check_att = d[d['att_check']==True]

In [ ]:
check_att

In [ ]:
correct = 0
for i in check_att['p_score']:
        correct += i
confusion_rate = (len(check_att) - correct)/len(check_att)
print(confusion_rate)

In [ ]:
for i in d:
        print(d[i].shape)

In [ ]:
len(set(d['model']))

In [ ]:
454 * 30 / 100

In [ ]:
no_check.groupby('model').mean()

In [ ]:
x = d.groupby('model').mean()

In [ ]:
x

In [ ]:
d

## Categorize data

In [ ]:
#stringcolumn in *args (order of *args) 
#e.g.: compare_model_sets(d,'models','subject')

def compare_model_sets(dataframe=d,*args): 
    #taking the sum of the values for dividing it later with the number of trials
    comparison = dataframe.groupby(by=[*args]).sum()
    ###print(comparison['p_score'])
    
    #counting the number of trials
    num = dataframe.groupby(by=[*args]).count()
    #print(num)
    #assign the number of trials to the relevant df
    comparison['num'] = num['p_score']
    #computing confusion rate from "p_score / num"
    #The confusion rate depends on how the grouping is put together
    comparison['confusion_rate'] = 1 - (comparison['p_score'] / num['p_score'])
    return comparison.reset_index()

#compute confusion_rate manually
def confusion_rate(p_score):
    correct = 0
    num = 0
    for i in p_score:
        correct += i
        num += 1
    return (num - correct)/num

In [ ]:
num = d.groupby(d['model']).count()
num

In [ ]:
num1 = d.groupby(d['model_set']).count()
num1

In [ ]:
compare_model_sets(d,'model')



In [ ]:
compare_model_sets(d,'model_set')

In [ ]:
dmp = d[d['model']=='dmp']
tmp = d[d['model']=='tmp']
v_elbo = d[d['model']=='vcgpdm-elbo']
v_map = d[d['model']=='vcgpdm-elbo']

In [ ]:
confusion_rate(no_check['p_score'])

In [ ]:
#df.groupby(by=['date', 'category']).sum()

In [ ]:
model = compare_model_sets(d,'model','model_set')
model

In [ ]:
model = compare_model_sets(d,'model_set','model')
model

In [ ]:
d['model_set'].unique()

In [ ]:
model = model.pivot_table(values='confusion_rate',index='model',columns='model_set').fillna(value=0)

In [ ]:
sns.heatmap(model)